layers.py : 각 층들을 구현한 코드 <br>
embedding : x가 one-hot이면 행렬곱을 하나 slicing을 하나 결과 같으므로 <b>계산비용 훨씬 싼<b> slicing 하자.<br>

# Embedding 층
/common/layers.py

In [ ]:
class Embedding:
    def __init__(self, W): # embedding 층은 input weight mat 주어져야 ... 따라서 초기화할 때 input weight mat W 줌. 
        self.params = [W] # 브라캣
        self.grads = [np.zeros_like(W)] # 그래디언트를 만들 때 (W로 손실함수를 미분하는 거니까) shape이 W와 같아야 하므로... W와 shape 같게 초기화
        self.idx = None

    def forward(self, idx): # 순전파, 배치처리 생각,,,, 단어들 몇 번 째 처음 등장,,, 단어들 idx,,, (1의 위치)
        W, = self.params
        self.idx = idx
        out = W[idx] # 슬라이싱 해라.
        return out # out을 forward로 보내라.

    def backward(self, dout): # 역전파 (dout: 흘러들어온 미분)
        dW, = self.grads # gradient는 브라켓 안에 있으니까 comma까지 해줘야 브라켓 안이 선택됨,,,
        dW[...] = 0 # [...]: shape을 맞춰줌., 몽땅 0으로 만들기... (zero mat)
        np.add.at(dW, self.idx, dout) # 흘러들어온 미분의 해당하는 행들 다 뽑아서 zero 행렬에 다 넣어서... 기존과 중복이면 더하기.(np.add.at(~~~))
                    # zero mat 미분, 위치, 흘러들어온 미분
        return None


# EmbeddingDot 층
/ch04/negative_sampling_layer.py (다음 강에서 오답도 sampling 해서 이진분류....)

In [ ]:
from common.layers import Embedding

class EmbeddingDot:
    def __init__(self, W): # 초기화, W: output Weight Mat를 Transpose 한 것... (원래는 해당하는 idx의 열 추출해야...
                                                                        # 근데 embedding 층은 input weight Mat의 행 추출... 우리는 embedding class 그대로 사용하고 싶으니까....
                                                                        # 열 뽑는 작업을 행 뽑는 작업으로 바꾸려면 Transpose...)
        self.embed = Embedding(W) # embedding class를 받는 embed 인스턴스 생성
        self.params = self.embed.params # embedding class에서 볼 수 있음. W에 브라켓
        self.grads = self.embed.grads # embedding class에서 볼 수 있음. Weight mat과 shape 동일한 zero mat에 브라켓
        self.cache = None

    def forward(self, h, idx): # 순전파를 하려면... h: 은닉층에 나타나는 [은닉 벡터] 또는 배치처리할 경우 [은닉 행렬] 필요,
                                                # idx: 입력하는 context의 중앙에 있는 target의 idx 모아둔 것. (slicing 하기 위해)
        target_W = self.embed.forward(idx) # forward method 적용 (forward: idx로 Weight mat slicing...)해서 target_W 변수에 저장...
        out = np.sum(target_W * h, axis=1) # *: 각 원소 그냥 곱하는 것,,,, scalar 곱 (상수배), np.sum(): 내적이니까 각 행 더해라...(?), axis=1: 열끼리 더하는게 행 합임...(가로로 뭉개서 더하는 거니까)

        self.cache = (h, target_W)
        return out # out을 다음으로 리턴

    def backward(self, dout): # 역전파, 흘러들어온 미분: 손실함수를 score로 미분한 것. (dL/ds)
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1) # dL/ds는 s와 shape이 동일해야 하므로.... 가로로 써진걸 세로로 바꿔라 뭐 이런....

        dtarget_W = dout * h # 미분하라. 손실함수를 target_W로(dL/dw) == 흘러들어온 미분과 은닉층 곱해라. (*: 상수배)
        self.embed.backward(dtarget_W) # backward method 적용,,, 일부분 slicing... 원래 같으면 행이 엄청 큰... # 이 줄 실행하는 순간 dW가 바뀜.
        dh = dout * target_W # 미분하라 손실함수를 h로 (dL/dh) == 흘러들어온 미분과 target_W (얘는 slicing 한 것...) 곱하기. (*: 상수배)
        return dh # 밑으로 흘려 보내야 하니까.... return...